# Neural Language Modeling

## 1. Data Processing
## 2. Encoder: Bag of Words
## 3. Decoder: MLP
## 4. Training Neural LM
## 5. Sentence Score
## 6. Sentence Generation

In [1]:
# !wget http://ikulikov.name/neural_lm_amazon_model_N10.pt
# !wget http://ikulikov.name/neural_lm_amazon_model_N7.pt
# !wget http://ikulikov.name/neural_lm_amazon_model_N5.pt
# !wget http://ikulikov.name/neural_lm_amazon_model_N3.pt

--2019-03-12 10:31:17--  http://ikulikov.name/neural_lm_amazon_model_N10.pt
Resolving ikulikov.name (ikulikov.name)... 173.255.226.199
Connecting to ikulikov.name (ikulikov.name)|173.255.226.199|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46933906 (45M) [application/octet-stream]
Saving to: ‘neural_lm_amazon_model_N10.pt’

neural_lm_amazon_mo 100%[===================>]  44.76M  12.1MB/s    in 4.2s    

2019-03-12 10:31:22 (10.6 MB/s) - ‘neural_lm_amazon_model_N10.pt’ saved [46933906/46933906]

--2019-03-12 10:31:22--  http://ikulikov.name/neural_lm_amazon_model_N7.pt
Resolving ikulikov.name (ikulikov.name)... 173.255.226.199
Connecting to ikulikov.name (ikulikov.name)|173.255.226.199|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46933906 (45M) [application/octet-stream]
Saving to: ‘neural_lm_amazon_model_N7.pt’

neural_lm_amazon_mo 100%[===================>]  44.76M  12.1MB/s    in 4.1s    

2019-03-12 10:31:27 (11.0 MB/s) - ‘neur

In [28]:
import os
import sys
sys.path.append('utils/')
import torch
import numpy as np
import torch.nn as nn
import random
import math

import utils.ngram_utils as ngram_utils
from utils.ngram_utils import NgramLM
from utils.amazon_dataset import AmazonDataset, pad, batchify
from torch.utils.data import DataLoader
from utils.neural_lm import BagOfNGrams, DecoderMLP, seq2seq
import utils.global_variables as gl
import torch
from tqdm import tqdm_notebook, tqdm
_tqdm = tqdm_notebook

In [2]:
torch.manual_seed(1)


In [3]:
USE_CUDA = True
device = torch.device("cuda" if (torch.cuda.is_available() and USE_CUDA) else "cpu")

# Data Processing

In [4]:
# Read data from .txt files and create lists of reviews
train_data = []
# create a list of all the reviews 
with open('../data/amazon_train.txt', 'r') as f:
    train_data = [review for review in f.read().split('\n') if review]
    
valid_data = []
# create a list of all the reviews 
with open('../data/amazon_valid.txt', 'r') as f:
    valid_data = [review for review in f.read().split('\n') if review]
    

In [5]:
train_data[0], valid_data[0]
train_data = train_data#[:100]
valid_data = valid_data#[:10]
train_data[0], type(train_data), len(train_data), type(train_data[0])

("this is a great tutu and at a really great price . it doesn ' t look cheap at all . i ' m so glad i looked on amazon and found such an affordable tutu that isn ' t made poorly . a + + ",
 list,
 22288,
 str)

In [6]:
# Tokenize the Datasets
# TODO: this takes a really long time !! why?
train_data_tokenized, all_tokens_train = ngram_utils.tokenize_dataset(train_data)
valid_data_tokenized, all_tokens_valid = ngram_utils.tokenize_dataset(valid_data)


In [7]:
N = 5

In [8]:
train_data_padded = ngram_utils.pad_dataset(train_data_tokenized, n=N)
valid_data_padded = ngram_utils.pad_dataset(valid_data_tokenized, n=N)

In [9]:
vocab = ngram_utils.get_vocab(train_data_padded)
vocab_size = len(vocab)
vocab_size, vocab[:10]

(20805, ('<eos>', '<sos>', '.', 'the', 'i', ',', 'and', 'a', 'to', "'"))

In [10]:
id2token, token2id = ngram_utils.get_dict(vocab)
len(id2token), len(token2id)

(20808, 20806)

In [11]:
train_data_ids = ngram_utils.get_ids(train_data_padded, token2id)
valid_data_ids = ngram_utils.get_ids(valid_data_padded, token2id)

In [12]:
BATCH_SIZE = 2048

In [13]:
train_dataset = AmazonDataset(train_data_ids, max_inp_length=None, use_cuda=USE_CUDA)
train_dataset_ngrams = []
for t in train_dataset:
    for i in range(len(t) - N):
        train_dataset_ngrams.append((t[i:i + N], t[i + N]))
train_loader = DataLoader(train_dataset_ngrams, batch_size=BATCH_SIZE, collate_fn=batchify, shuffle=True)

100%|██████████| 107790/107790 [00:06<00:00, 16200.06it/s]


In [14]:
valid_dataset = AmazonDataset(valid_data_ids, max_inp_length=None, use_cuda=USE_CUDA)
valid_dataset_ngrams = []
for t in valid_dataset:
    for i in range(len(t) - N):
        valid_dataset_ngrams.append((t[i:i + N], t[i + N]))
valid_loader = DataLoader(valid_dataset_ngrams, batch_size=BATCH_SIZE, collate_fn=batchify, shuffle=True)

100%|██████████| 15172/15172 [00:00<00:00, 29778.23it/s]


In [15]:
num_train = len(train_dataset_ngrams)
num_valid = len(valid_dataset_ngrams)
num_train, num_valid

(1946816, 280739)

In [16]:
EMB_DIM = 300
HIDDEN_SIZE = 256
ENC_SIZE = 128
ACTIVATION = 'ReLU'
NUM_LAYERS = 2
REDUCE = 'mean'
DROPOUT = 0.1
LEARNING_RATE = 0.1
BATCH_NORM = False

# Encoder

In [17]:
encoder = BagOfNGrams(len(id2token), emb_dim=EMB_DIM, hidden_size=HIDDEN_SIZE, out_size=ENC_SIZE, activation=ACTIVATION, \
                        nlayers=NUM_LAYERS, reduce=REDUCE, dropout=DROPOUT, batch_norm=BATCH_NORM)
encoder

BagOfNGrams(
  (embedding): EmbeddingBag(20808, 300, mode=mean)
  (layers): ModuleList(
    (0): Linear(in_features=300, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.1)
    (3): Linear(in_features=256, out_features=128, bias=True)
  )
)

# Decoder

In [18]:
decoder = DecoderMLP(input_size=ENC_SIZE, output_size=len(id2token), hidden_size=HIDDEN_SIZE)
decoder

DecoderMLP(
  (linear): Linear(in_features=128, out_features=256, bias=True)
  (out): Linear(in_features=256, out_features=20808, bias=True)
  (log_softmax): LogSoftmax()
)

# Model

In [19]:
model = seq2seq(encoder, decoder, id2token, use_cuda=USE_CUDA, lr=LEARNING_RATE, size_ngrams=N) 
model

seq2seq(
  (encoder): BagOfNGrams(
    (embedding): EmbeddingBag(20808, 300, mode=mean)
    (layers): ModuleList(
      (0): Linear(in_features=300, out_features=256, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.1)
      (3): Linear(in_features=256, out_features=128, bias=True)
    )
  )
  (decoder): DecoderMLP(
    (linear): Linear(in_features=128, out_features=256, bias=True)
    (out): Linear(in_features=256, out_features=20808, bias=True)
    (log_softmax): LogSoftmax()
  )
  (criterion): NLLLoss()
)

In [20]:
TRAIN = False

# Training

In [ ]:
if TRAIN:
    num_epochs = 10
    log_interval = 1000
    best_eval_loss = np.inf

    for epoch in range(num_epochs):
        # Train on entire train data using minibatches
        cur_loss = 0
        for i, (data, labels) in enumerate(train_loader):
            prediction, loss = model.train_step(data, labels)
            cur_loss += loss

            if i % log_interval == 0 and i > 0:
                cur_loss = cur_loss / log_interval
                print('| Epoch {:3d} | Train Loss {:5.2f} | Train PPL {:8.2f} | {:5d}/{:5d} Batches'.format(
                    epoch, cur_loss, math.exp(cur_loss), i, int(num_train/len(data))))
                cur_loss = 0

        # Eval at the end of every epoch
        eval_loss = 0
        for i, (data, labels) in enumerate(valid_loader):
            prediction, loss = model.eval_step(data, labels)
            eval_loss += len(data) * loss
        eval_loss = eval_loss / num_valid 
        print('-' * 89)
        print('| Epoch {:3d} | Valid Loss {:5.2f} | Valid PPL {:8.2f}'.format(
            epoch, eval_loss, math.exp(eval_loss)))
        print('-' * 89)

        # Save the model if the validation loss is the best we've seen so far.
        if not best_eval_loss or eval_loss < best_eval_loss:
            model.save_model('neural_lm_amazon_model_N{}'.format(N) + '.pt')
            best_eval_loss = eval_loss
            
    pretrained_model = model

else:
    # Load Pretrained Model
    pretrained_model = seq2seq(encoder, decoder, id2token, use_cuda=USE_CUDA, lr=LEARNING_RATE, size_ngrams=N) 
    pretrained_model.load_model('neural_lm_amazon_model_N{}'.format(N) + '.pt')
    pretrained_model

| Epoch   0 | Train Loss  4.69 | Train PPL   109.34 |  1000/ 9386 Batches
| Epoch   0 | Train Loss  4.13 | Train PPL    62.38 |  2000/ 9386 Batches
| Epoch   0 | Train Loss  4.04 | Train PPL    56.74 |  3000/ 9386 Batches
| Epoch   0 | Train Loss  3.99 | Train PPL    54.11 |  4000/ 9386 Batches
| Epoch   0 | Train Loss  3.95 | Train PPL    52.07 |  5000/ 9386 Batches
| Epoch   0 | Train Loss  3.93 | Train PPL    51.02 |  6000/ 9386 Batches
| Epoch   0 | Train Loss  3.91 | Train PPL    49.67 |  7000/ 9386 Batches
| Epoch   0 | Train Loss  3.89 | Train PPL    48.86 |  8000/ 9386 Batches
| Epoch   0 | Train Loss  3.88 | Train PPL    48.32 |  9000/ 9386 Batches
-----------------------------------------------------------------------------------------
| Epoch   0 | Valid Loss  3.68 | Valid PPL    39.72
-----------------------------------------------------------------------------------------
| Epoch   1 | Train Loss  3.85 | Train PPL    47.07 |  1000/ 9386 Batches
| Epoch   1 | Train Loss  3.

### Get Loader for Some N

In [21]:
def get_loader(train_data_tokenized, valid_data_tokenized, N):
    # Tokenize the Datasets
    # TODO: this takes a really long time !! why?
    train_data_tokenized, all_tokens_train = ngram_utils.tokenize_dataset(train_data)
    valid_data_tokenized, all_tokens_valid = ngram_utils.tokenize_dataset(valid_data)
    
    train_data_padded = ngram_utils.pad_dataset(train_data_tokenized, n=N)
    valid_data_padded = ngram_utils.pad_dataset(valid_data_tokenized, n=N)
    
    vocab = ngram_utils.get_vocab(train_data_padded)
    id2token, token2id = ngram_utils.get_dict(vocab)
    
    train_data_ids = ngram_utils.get_ids(train_data_padded, token2id)
    valid_data_ids = ngram_utils.get_ids(valid_data_padded, token2id)

    train_dataset = AmazonDataset(train_data_ids, max_inp_length=None, use_cuda=USE_CUDA)
    train_dataset_ngrams = []
    for t in train_dataset:
        for i in range(len(t) - N):
            train_dataset_ngrams.append((t[i:i + N], t[i + N]))
    train_loader = DataLoader(train_dataset_ngrams, batch_size=BATCH_SIZE, collate_fn=batchify, shuffle=True)

    valid_dataset = AmazonDataset(valid_data_ids, max_inp_length=None, use_cuda=USE_CUDA)
    valid_dataset_ngrams = []
    for t in valid_dataset:
        for i in range(len(t) - N):
            valid_dataset_ngrams.append((t[i:i + N], t[i + N]))
    valid_loader = DataLoader(valid_dataset_ngrams, batch_size=BATCH_SIZE, collate_fn=batchify, shuffle=True)
    
    num_train = len(train_dataset_ngrams)
    num_valid = len(valid_dataset_ngrams)
    
    return train_loader, num_train, valid_loader, num_valid

## Perplexity (Train + Valid Data)

In [22]:
def get_perplexity(loader, num_data, model):
    total_loss = 0
    total_count = 0
    for i, (data, labels) in enumerate(loader):
        prediction, loss = model.eval_step(data, labels)
        total_loss += len(data) * loss
        total_count += len(data)
    total_loss = total_loss / total_count
    ppl = math.exp(total_loss)
    return ppl

In [23]:
# Load Pretrained Model
N = 3
print("Ngrams: ", N)

model_n3 = seq2seq(encoder, decoder, id2token, use_cuda=USE_CUDA, lr=LEARNING_RATE, size_ngrams=N) 
model_n3.load_model('neural_lm_amazon_model_N{}'.format(N) + '.pt')
print("Model: ", model_n3)

train_loader, num_train, valid_loader, num_valid = get_loader(train_data_tokenized, valid_data_tokenized, N)

valid_ppl = get_perplexity(valid_loader, num_valid, model_n3)
print("Validation Perplexity: ", valid_ppl)

train_ppl = get_perplexity(train_loader, num_train, model_n3)
print("Training Perplexity: ", train_ppl)


Ngrams:  3
Model:  seq2seq(
  (encoder): BagOfNGrams(
    (embedding): EmbeddingBag(20808, 300, mode=mean)
    (layers): ModuleList(
      (0): Linear(in_features=300, out_features=256, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.1)
      (3): Linear(in_features=256, out_features=128, bias=True)
    )
  )
  (decoder): DecoderMLP(
    (linear): Linear(in_features=128, out_features=256, bias=True)
    (out): Linear(in_features=256, out_features=20808, bias=True)
    (log_softmax): LogSoftmax()
  )
  (criterion): NLLLoss()
)


100%|██████████| 15172/15172 [00:00<00:00, 30064.85it/s]


Validation Perplexity:  132.2111764559884
Training Perplexity:  117.42600925705602


In [32]:
# Load Pretrained Model
N = 5
print("Ngrams: ", N)

model_n5 = seq2seq(encoder, decoder, id2token, use_cuda=USE_CUDA, lr=LEARNING_RATE, size_ngrams=N) 
model_n5.load_model('neural_lm_amazon_model_N{}'.format(N) + '.pt')
print("Model: ", model_n5)

train_loader, num_train, valid_loader, num_valid = get_loader(train_data_tokenized, valid_data_tokenized, N)

valid_ppl = get_perplexity(valid_loader, num_valid, model_n5)
print("Validation Perplexity: ", valid_ppl)

train_ppl = get_perplexity(train_loader, num_train, model_n5)
print("Training Perplexity: ", train_ppl)


Ngrams:  5
Model:  seq2seq(
  (encoder): BagOfNGrams(
    (embedding): EmbeddingBag(20808, 300, mode=mean)
    (layers): ModuleList(
      (0): Linear(in_features=300, out_features=256, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.1)
      (3): Linear(in_features=256, out_features=128, bias=True)
    )
  )
  (decoder): DecoderMLP(
    (linear): Linear(in_features=128, out_features=256, bias=True)
    (out): Linear(in_features=256, out_features=20808, bias=True)
    (log_softmax): LogSoftmax()
  )
  (criterion): NLLLoss()
)


100%|██████████| 15172/15172 [00:00<00:00, 30075.63it/s]


Validation Perplexity:  80.90786768026997
Training Perplexity:  74.61620960861192


In [ ]:
# Load Pretrained Model
N = 7
print("Ngrams: ", N)

model_n7 = seq2seq(encoder, decoder, id2token, use_cuda=USE_CUDA, lr=LEARNING_RATE, size_ngrams=N) 
model_n7.load_model('neural_lm_amazon_model_N{}'.format(N) + '.pt')
print("Model: ", model_n7)

train_loader, num_train, valid_loader, num_valid = get_loader(train_data_tokenized, valid_data_tokenized, N)

valid_ppl = get_perplexity(valid_loader, num_valid, model_n7)
print("Validation Perplexity: ", valid_ppl)

train_ppl = get_perplexity(train_loader, num_train, model_n7)
print("Training Perplexity: ", train_ppl)


In [28]:
# Load Pretrained Model
N = 10
print("Ngrams: ", N)

model_n10 = seq2seq(encoder, decoder, id2token, use_cuda=USE_CUDA, lr=LEARNING_RATE, size_ngrams=N) 
model_n10.load_model('neural_lm_amazon_model_N{}'.format(N) + '.pt')
print("Model: ", model_n10)

train_loader, num_train, valid_loader, num_valid = get_loader(train_data_tokenized, valid_data_tokenized, N)

valid_ppl = get_perplexity(valid_loader, num_valid, model_n10)
print("Validation Perplexity: ", valid_ppl)

train_ppl = get_perplexity(train_loader, num_train, model_n10)
print("Training Perplexity: ", train_ppl)


Validation Perplexity:  33.31333717417352
Training Perplexity:  29.76822279473455


## Score Sentences

In [29]:
def score_sentence(sent, model):
    tokenized, _ = ngram_utils.tokenize_dataset(sent)
    sent_ids = ngram_utils.get_ids(tokenized, token2id)
    sent_tensor = torch.LongTensor(sent_ids).to(device)
    generated, scores = model.evaluate(sent_tensor, score_only=True)
    ppl = math.exp(scores)
    return ppl

In [30]:
sentence = ['i like this product very much .']
ppl3 = score_sentence(sentence, model_n3)
ppl5 = score_sentence(sentence, model_n5)
ppl7 = score_sentence(sentence, model_n7)
ppl10 = score_sentence(sentence, model_n10)
ppl3, ppl5, ppl7, ppl10

RuntimeError: Expected object of backend CUDA but got backend CPU for sequence element 1 in sequence argument at position #1 'tensors'

In [ ]:
sentence = ['i like pandas']
ppl3 = score_sentence(sentence, model_n3)
ppl5 = score_sentence(sentence, model_n5)
ppl7 = score_sentence(sentence, model_n7)
ppl10 = score_sentence(sentence, model_n10)
ppl3, ppl5, ppl7, ppl10

In [ ]:
sentences = ['this color is very ugly !']
ppl3 = score_sentence(sentence, model_n3)
ppl5 = score_sentence(sentence, model_n5)
ppl7 = score_sentence(sentence, model_n7)
ppl10 = score_sentence(sentence, model_n10)
ppl3, ppl5, ppl7, ppl10

In [ ]:
sentences = ['kigali is an awesome city !']
ppl3 = score_sentence(sentence, model_n3)
ppl5 = score_sentence(sentence, model_n5)
ppl7 = score_sentence(sentence, model_n7)
ppl10 = score_sentence(sentence, model_n10)
ppl3, ppl5, ppl7, ppl10

In [ ]:
sentences = ['my wife loves the color of this dress']
ppl3 = score_sentence(sentence, model_n3)
ppl5 = score_sentence(sentence, model_n5)
ppl7 = score_sentence(sentence, model_n7)
ppl10 = score_sentence(sentence, model_n10)
ppl3, ppl5, ppl7, ppl10

In [ ]:
sentence = ['i like pandas']
ppl3 = score_sentence(sentence, model_n3)
ppl5 = score_sentence(sentence, model_n5)
ppl7 = score_sentence(sentence, model_n7)
ppl10 = score_sentence(sentence, model_n10)
ppl3, ppl5, ppl7, ppl10

RuntimeError: Expected object of backend CUDA but got backend CPU for sequence element 1 in sequence argument at position #1 'tensors'

## Generate Sentences

In [ ]:
def generate_sentence(model, context=None):
    if context is None:
        dummy_context = torch.LongTensor([[0]]).to(device)
        generated, scores = model.evaluate(dummy_context, use_context=False)
    else:
        tokenized, _ = ngram_utils.tokenize_dataset(context)
        context_ids = ngram_utils.get_ids(tokenized, token2id)
        context_tensor = torch.LongTensor(context_ids).to(device)
        generated, scores = model.evaluate(context_tensor, use_context=True)
    
    ppl = math.exp(scores)
    return generated, scores

### No Context

In [ ]:
generated, scores = generate_sentence(model_n3)
print(' '.join(word[0] for word in generated)), scores

In [ ]:
generated, scores = generate_sentence(model_n5)
print(' '.join(word[0] for word in generated))

In [ ]:
generated, scores = generate_sentence(model_n7)
print(' '.join(word[0] for word in generated))

In [ ]:
generated, scores = generate_sentence(model_n10)
print(' '.join(word[0] for word in generated))

### With Context

In [ ]:
generated, scores = generate_sentence(model_n3, context=['this dress'])
print(' '.join(word[0] for word in generated))

In [ ]:
generated, scores = generate_sentence(model_n5, context=['this dress'])
print(' '.join(word[0] for word in generated))

In [ ]:
generated, scores = generate_sentence(model_n7, context=['this dress'])
print(' '.join(word[0] for word in generated))

In [ ]:
generated, scores = generate_sentence(model_n10, context=['this dress'])
print(' '.join(word[0] for word in generated))

In [ ]:
generated, scores = generate_sentence(model_n3, context=['i like'])
print(' '.join(word[0] for word in generated))

In [ ]:
generated, scores = generate_sentence(model_n5, context=['i like'])
print(' '.join(word[0] for word in generated))

In [ ]:
generated, scores = generate_sentence(model_n7, context=['i like'])
print(' '.join(word[0] for word in generated))

In [ ]:
generated, scores = generate_sentence(model_n10, context=['i like'])
print(' '.join(word[0] for word in generated))